In [ ]:
import pandas as pd
import monai
import json
import nibabel as nib

from monai.transforms import (
    AsDiscrete,
    CropForegroundd,
    RandShiftIntensityd,
    RandAffined,
    Compose,
    LoadImaged,
    Orientationd,
    ScaleIntensityRanged,
    Spacingd,
    EnsureChannelFirstd,
    SpatialPadd,
    ToTensord,
    RandCropByPosNegLabeld,
    RandScaleIntensityd,
    RandGaussianNoised,
    RandAdjustContrastd,
    RandFlipd,
    RandGaussianSmoothd,
    RandRotated,
)
import numpy as np

In [35]:
#Read Data
df = pd.read_csv('Prostate_Cancer_TFM/Files/PICAI/partition_1_cv.csv')

#Read folds and split data
json_path = "/nvmescratch/ceib/Prostate/workdir/splits/picai_nnunet/splits.json"
with open(json_path, "r") as f:
    splits = json.load(f)

#Get the first fold
for i,fold in enumerate(splits):
    train = fold['train']
    val = fold['val']
    
    subjects_to_remove = ['10251_1000255', '10319_1000325', '11050_1001070', '11214_1001237', '11261_1001284','10233_1000237','11032_1001052']
    train = [x for x in train if x not in subjects_to_remove]
    val = [x for x in val if x not in subjects_to_remove]


    df['index'] = df['patient_id'].astype(str)+'_'+df['study_id'].astype(str)
    df.set_index('index', inplace=True)

    df.loc[train,f'Fold {i}']='train'
    df.loc[val,f'Fold {i}']='val'

In [36]:
df

,patient_id,study_id,mri_date,patient_age,psa,psad,prostate_volume,histopath_type,lesion_GS,lesion_ISUP,...,filepath_labelAI_cropped,filepath_seg_zones,filepath_seg_zones_cropped,PIRADS,cv_partition,Fold 0,Fold 1,Fold 2,Fold 3,Fold 4
index,,,,,,,,,,,,,,,,,,,,,
10347_1000353,10347,1000353,2020-07-15,58,9.8,0.17,58.0,SysBx+MRBx,"0+0,0+0","0,0",...,/nvmescratch/ceib/Prostate/input/cropped/label...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/labels/anatom...,NaN,1,train,train,train,val,train
10166_1000169,10166,1000169,2019-10-09,68,4.3,0.20,21.0,NaN,NaN,NaN,...,/nvmescratch/ceib/Prostate/input/cropped/label...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/labels/anatom...,NaN,2,train,train,train,train,val
11309_1001332,11309,1001332,2012-03-04,50,7.5,0.34,22.0,MRBx,0+0,0,...,/nvmescratch/ceib/Prostate/input/cropped/label...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/labels/anatom...,NaN,1,train,val,train,train,train
11000_1001019,11000,1001019,2019-01-11,51,1.2,0.03,33.0,NaN,NaN,NaN,...,/nvmescratch/ceib/Prostate/input/cropped/label...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/labels/anatom...,NaN,2,train,train,val,train,train
10173_1000176,10173,1000176,2016-11-25,64,28.0,NaN,37.0,SysBx,0+0,0,...,/nvmescratch/ceib/Prostate/input/cropped/label...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/labels/anatom...,NaN,2,val,train,train,train,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11327_1001350,11327,1001350,2012-06-18,49,3.0,0.10,29.0,MRBx,"0+0,0+0","0,0",...,/nvmescratch/ceib/Prostate/input/cropped/label...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/labels/anatom...,NaN,3,train,train,val,train,train
10398_1000404,10398,1000404,2012-08-07,62,33.0,0.52,64.0,MRBx,3+4,2,...,/nvmescratch/ceib/Prostate/input/cropped/label...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/labels/anatom...,2.0,3,train,train,train,val,train
10343_1000349,10343,1000349,2012-08-24,67,21.9,0.23,92.0,MRBx,"3+2,N/A",1,...,/nvmescratch/ceib/Prostate/input/cropped/label...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/labels/anatom...,NaN,4,train,val,train,train,train


In [37]:
#Check images

fold = 0

format_load = "cropped"  #'nifti', mha, cropped
df["depth"] = df["filepath_t2w_" + format_load].apply(lambda path_file: nib.load(path_file).shape[0])
df["heigth"] = df["filepath_t2w_" + format_load].apply(lambda path_file: nib.load(path_file).shape[1])
df["weigth"] = df["filepath_t2w_" + format_load].apply(lambda path_file: nib.load(path_file).shape[2])
df = df[(df["heigth"] != 0) & (df["depth"] != 0)]
df = df[df["filepath_t2w_" + format_load].notna()].reset_index()

df = df[(df["heigth"] > 96) & (df["depth"] > 96)]

data_picai = df

data_picai_human = data_picai[data_picai["human_labeled"] == 1]
data_picai.drop(data_picai_human.index, inplace=True)

data_picai = data_picai[
    [
        "filepath_t2w_" + format_load,
        "filepath_adc_" + format_load,
        "filepath_hbv_" + format_load,
        "filepath_labelAI_cropped",
        "filepath_seg_zones_cropped",
        f'Fold {fold}',
    ]
]
data_picai_human = data_picai_human[
    [
        "filepath_t2w_" + format_load,
        "filepath_adc_" + format_load,
        "filepath_hbv_" + format_load,
        "filepath_label_cropped",
        "filepath_seg_zones_cropped",
        f'Fold {fold}',
    ]
]

df = pd.DataFrame(
    {
        "t2w": list(data_picai["filepath_t2w_" + format_load].values)
        + list(data_picai_human["filepath_t2w_" + format_load].values),
        "adc": list(data_picai["filepath_adc_" + format_load].values)
        + list(data_picai_human["filepath_adc_" + format_load].values),
        "dwi": list(data_picai["filepath_hbv_" + format_load].values)
        + list(data_picai_human["filepath_hbv_" + format_load].values),
        "zones": list(data_picai["filepath_seg_zones_cropped"].values) + list(data_picai_human["filepath_seg_zones_cropped"].values),
        "label": list(data_picai["filepath_labelAI_cropped"].values) + list(data_picai_human["filepath_label_cropped"].values),
        "partition": list(data_picai[f'Fold {fold}'].values) + list(data_picai_human[f'Fold {fold}'].values),
    }
)

In [38]:
df

,t2w,adc,dwi,zones,label,partition
0,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/cropped/label...,train
1,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/cropped/label...,train
2,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/cropped/label...,train
3,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/cropped/label...,train
4,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/cropped/label...,val
...,...,...,...,...,...,...
1486,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/cropped/label...,train
1487,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/cropped/label...,train
1488,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/cropped/label...,train
1489,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/cropped/image...,/nvmescratch/ceib/Prostate/input/labels/anatom...,/nvmescratch/ceib/Prostate/input/cropped/label...,train


In [ ]:
patch_size = [256, 160, 224]
img_columns=["t2", "adc", "dwi"]
label_column = ["label"]

train_transforms = Compose(
    [
        LoadImaged(keys=img_columns + label_column + ["zones"], reader="NibabelReader", image_only=True),
        AsDiscreted(keys=label_column, threshold=1),  # Convert values greater than 1 to 1
        EnsureChannelFirstd(keys=img_columns + label_column + ["zones"]),
        AsDiscreted(keys="zones", argmax=False, to_onehot=3),
        LabelToMaskd(keys="zones", select_labels=[1, 2]),
        ResampleToMatchd(
            keys=["adc", "dwi", "zones", "label"], key_dst="t2", mode=("bilinear", "bilinear", "nearest", "nearest")
        ),
        EnsureChannelFirstd(keys=["image", "label"], channel_dim="no_channel"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 1.5),
            mode=("bilinear", "nearest"),
        ),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-982,
            a_max=1094,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        SpatialPadd(keys=["image", "label"], spatial_size=patch_size),
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=patch_size,
            pos=2,
            neg=1,
            num_samples=args["num_samples_per_image"],
            image_key="image",
            image_threshold=0,
        ),
        RandRotated(
            keys=["image", "label"],
            prob=0.2,
            range_x=(-30.0 / 360 * 2.0 * np.pi, 30.0 / 360 * 2.0 * np.pi),
            range_y=(-30.0 / 360 * 2.0 * np.pi, 30.0 / 360 * 2.0 * np.pi),
            range_z=(-30.0 / 360 * 2.0 * np.pi, 30.0 / 360 * 2.0 * np.pi),
            mode=["bilinear", "nearest"],
        ),
        RandScaleIntensityd(keys=["image"], prob=0.2, factors=(0.7, 1.4)),
        RandGaussianNoised(keys="image", prob=0.1, mean=0, std=0.1),
        RandGaussianSmoothd(keys="image", prob=0.1, sigma_x=(0.5, 1)),
        RandAdjustContrastd(keys="image", prob=0.15, gamma=(0.75, 1.25)),
        RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=[0]),
        RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=[1]),
        RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=[2]),
        ToTensord(keys=["image", "label"]),
    ]
)

val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"], channel_dim="no_channel"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 1.5),
            mode=("bilinear", "nearest"),
        ),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-982,
            a_max=1094,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        SpatialPadd(keys=["image", "label"], spatial_size=args["patch_size"]),
        ToTensord(keys=["image", "label"]),
    ]
)

